In [1]:
import numpy as np
import torchvision
import time
import os
import copy
import pdb
import time
import argparse

import sys
import cv2

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms

from dataloader import CocoDataset, CSVDataset, collater,\
Resizer, AspectRatioBasedSampler, Augmenter, UnNormalizer, Normalizer


assert torch.__version__.split('.')[1] == '4'

print('CUDA available: {}'.format(torch.cuda.is_available()))


CUDA available: True


In [2]:
dataset_val = CSVDataset(train_file='val_csv/imgm_Patient3_Normal_test.csv', class_list='octID.csv',\
                         transform=transforms.Compose([Normalizer(), Resizer()]))

In [3]:
sampler_val = AspectRatioBasedSampler(dataset_val, batch_size=1, drop_last=False)
dataloader_val = DataLoader(dataset_val, num_workers=1, collate_fn=collater, batch_sampler=sampler_val)

retinanet = torch.load('Results_Jan24_2019/model_Jan24.pt')

use_gpu = True

if use_gpu:
    retinanet = retinanet.cuda()

retinanet.eval()

unnormalize = UnNormalizer()

print(len(dataloader_val))

600


In [4]:
def draw_caption(image, box, caption):

    b = np.array(box).astype(int)
    cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 2)
    cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)

In [5]:
imagOutpath = 'imageOutput'
for idx, data in enumerate(dataloader_val):

    with torch.no_grad():
        st = time.time()
        scores, classification, transformed_anchors = retinanet(data['img'].cuda().float())
        print('Elapsed time: {}'.format(time.time()-st))
        idxs = np.where(scores>0.5)
        img = np.array(255 * unnormalize(data['img'][0, :, :, :])).copy()

        img[img<0] = 0
        img[img>255] = 255
        print(img.shape)
        
        
        

        img = np.transpose(img, (1, 2, 0))
        imgsave = img

        img = cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_BGR2RGB)
        #cv2.imshow('img', img)




        for j in range(idxs[0].shape[0]):
            bbox = transformed_anchors[idxs[0][j], :]
            x1 = int(bbox[0])
            y1 = int(bbox[1])
            x2 = int(bbox[2])
            y2 = int(bbox[3])
            label_name = dataset_val.labels[int(classification[idxs[0][j]])]
            if label_name == 'teeth':
                draw_caption(img, (x1, y1, x2, y2), label_name)

                mycolor = [255,0,0]


                cv2.rectangle(img, (x1, y1), (x2, y2), color=(mycolor), thickness=2)
            #print(label_name)

        #print(img.shape)
        #filestring = 'imgm_Patient115_Normal_test'+str(idx)+'.jpg'
        #print(filestring)
        #cv2.imwrite(os.path.join(imagOutpath, filestring), img)
        #cv2.imshow('img', img)
        cv2.waitKey(0)

/home/lazybaby7/anaconda3/envs/torchRetina/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/lazybaby7/anaconda3/envs/torchRetina/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Elapsed time: 0.04448437690734863
(3, 640, 640)


/home/lazybaby7/anaconda3/envs/torchRetina/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


Elapsed time: 0.040845632553100586
(3, 640, 640)
Elapsed time: 0.03579354286193848
(3, 640, 640)
Elapsed time: 0.03951549530029297
(3, 640, 640)
Elapsed time: 0.03954362869262695
(3, 640, 640)
Elapsed time: 0.03480124473571777
(3, 640, 640)
Elapsed time: 0.03964591026306152
(3, 640, 640)
Elapsed time: 0.039077043533325195
(3, 640, 640)
Elapsed time: 0.03698921203613281
(3, 640, 640)
Elapsed time: 0.037796974182128906
(3, 640, 640)
Elapsed time: 0.04010272026062012
(3, 640, 640)
Elapsed time: 0.03767538070678711
(3, 640, 640)
Elapsed time: 0.038813114166259766
(3, 640, 640)
Elapsed time: 0.03775787353515625
(3, 640, 640)
Elapsed time: 0.03838181495666504
(3, 640, 640)
Elapsed time: 0.04073023796081543
(3, 640, 640)
Elapsed time: 0.03947257995605469
(3, 640, 640)
Elapsed time: 0.036199092864990234
(3, 640, 640)
Elapsed time: 0.040059566497802734
(3, 640, 640)
Elapsed time: 0.03842496871948242
(3, 640, 640)
Elapsed time: 0.03611421585083008
(3, 640, 640)
Elapsed time: 0.03843116760253906


Process Process-1:
Traceback (most recent call last):
  File "/home/lazybaby7/anaconda3/envs/torchRetina/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/lazybaby7/anaconda3/envs/torchRetina/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/lazybaby7/anaconda3/envs/torchRetina/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
KeyboardInterrupt
  File "/home/lazybaby7/anaconda3/envs/torchRetina/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/lazybaby7/workSpace/pytorch-retinanet/dataloader.py", line 207, in __getitem__
    img = self.load_image(idx)
  File "/home/lazybaby7/workSpace/pytorch-retinanet/dataloader.py", line 219, in load_image
    img = skimage.color.g

KeyboardInterrupt: 

In [ ]:
cv2.imshow('img',imgsave)
cv2.waitKey(0)

In [ ]:
type(imgsave)

In [ ]:
type(img)